In [13]:
import ee
import geemap


ee.Authenticate()

ee.Initialize(project='ee-edmaurogoma2000')


In [14]:
# Define the variables
region = [10, 20, 30, 40]  # Coordinates of the region
collection = 'MODIS/006/MOD44B'  # Name of the MODIS image collection
date_range = ['2020-01-01', '2020-03-05']  # Date range for filtering images

# Create an Earth Engine geometry from coordinates
geometry = ee.Geometry.Rectangle([region[0], region[1], region[2], region[3]])

# Create a MODIS image collection
dataset = (ee.ImageCollection(collection)
           .filterBounds(geometry)
           .filterDate(date_range[0], date_range[1]))

# Function to add a timestamp band to each image
def add_timestamp(image):
    return image.addBands(image.metadata('system:time_start').rename('timestamp'))

# Map the function over the image collection
dataset_with_timestamp = dataset.map(add_timestamp)

# Get the size of the collection
collection_size = dataset_with_timestamp.size().getInfo()
print(f"Number of images in the collection: {collection_size}")

# Define a function to export an image
def export_image(image, filename):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        folder='Earth_Engine_Exports',
        fileNamePrefix=filename,
        scale=500,
        region=geometry,
        maxPixels=1e9
    )
    task.start()
    return task

# Export images (limit to first 10 for demonstration)
tasks = []
for i in range(min(10, collection_size)):
    image = ee.Image(dataset_with_timestamp.toList(10).get(i))
    filename = f'modis_image_{i:03d}'
    task = export_image(image, filename)
    tasks.append(task)

print("Export tasks have been started. Check your Google Drive 'Earth_Engine_Exports' folder for results.")

# Optional: Display the first image on a map
Map = geemap.Map()
Map.centerObject(geometry, 6)
Map.addLayer(ee.Image(dataset_with_timestamp.first()), {'bands': ['Percent_Tree_Cover']}, 'Tree Cover')
Map.addLayer(geometry, {'color': 'FF0000'}, 'ROI')
display(Map)

Number of images in the collection: 0
Export tasks have been started. Check your Google Drive 'Earth_Engine_Exports' folder for results.


EEException: Image.visualize: Parameter 'image' is required.